In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input,Embedding,Flatten,Dot,Dense,Dropout,Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [3]:
num_users = 500
num_items = 150
ratings = np.random.randint(0,6,size=(num_users,num_items))
ratings.shape

(500, 150)

In [4]:
train_ratings , test_ratings = train_test_split(ratings,test_size=0.2)
print(train_ratings.shape)
print(test_ratings.shape)

(400, 150)
(100, 150)


In [5]:
embedding_size = 50
user_input = Input(shape=(1,))
item_input = Input(shape=(1,))

In [6]:
user_embedding = Embedding(input_dim=num_users,output_dim = embedding_size)(user_input)
item_embedding = Embedding(input_dim=num_items,output_dim = embedding_size)(item_input)

In [7]:
user_flat = Flatten()(user_embedding)
item_flat = Flatten()(item_embedding)

In [8]:
concat = Concatenate()([user_flat,item_flat])

dense1 = Dense(128,activation='relu')(concat)
dense2 = Dense(64,activation='relu')(dense1)


In [9]:
model = Model(
    inputs = [user_input,item_input],
    outputs = Dense(1,activation="relu")(dense2)
)

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 embedding (Embedding)       (None, 1, 50)                25000     ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, 1, 50)                7500      ['input_2[0][0]']             
                                                                                              

In [11]:
model.compile(
  loss="binary_crossentropy",
  optimizer = Adam(learning_rate=0.001),
  metrics = ["accuracy"]
)

In [12]:
model.fit(
    x = [train_ratings[:,0],train_ratings[:,1]],
    y = train_ratings[:,2],
    batch_size=32,
    epochs=250,
    validation_split=0.2
)

Epoch 1/250
10/10 [==============================] - 2s 59ms/step - loss: 8.1336 - accuracy: 0.1906 - val_loss: 3.5894 - val_accuracy: 0.1250
Epoch 2/250
10/10 [==============================] - 0s 14ms/step - loss: 2.3675 - accuracy: 0.1969 - val_loss: 0.9593 - val_accuracy: 0.1250
Epoch 3/250
10/10 [==============================] - 0s 16ms/step - loss: -1.2230 - accuracy: 0.1500 - val_loss: -11.0293 - val_accuracy: 0.1375
Epoch 4/250
10/10 [==============================] - 0s 12ms/step - loss: -17.6401 - accuracy: 0.1437 - val_loss: -23.4457 - val_accuracy: 0.1375
Epoch 5/250
10/10 [==============================] - 0s 9ms/step - loss: -22.2067 - accuracy: 0.1437 - val_loss: -23.4457 - val_accuracy: 0.1375
Epoch 6/250
10/10 [==============================] - 0s 10ms/step - loss: -22.2067 - accuracy: 0.1437 - val_loss: -23.4457 - val_accuracy: 0.1375
Epoch 7/250
10/10 [==============================] - 0s 8ms/step - loss: -22.2067 - accuracy: 0.1437 - val_loss: -23.4457 - val_accura

In [13]:
loss,accuracy = model.evaluate(x = [train_ratings[:,0],train_ratings[:,1]],y = train_ratings[:,2])
print(loss)
print(accuracy*100)

13/13 [==============================] - 0s 2ms/step - loss: -22.4545 - accuracy: 0.1425
-22.45450210571289
14.249999821186066


In [20]:
def recommedor(user_id):
  user_ids = np.full(num_items,user_id)
  item_ids = np.arange(num_items)
  ratings = model.predict([user_ids,item_ids])
  # print(ratings)
  top_five = np.argpartition(ratings.flatten(), 10)[:10]
  # print(np.argpartition(ratings.flatten(), 5))
  return top_five

In [21]:
user_id = int(input("Enter the User Id : "))
result = recommedor(user_id)
print(result)

# [100  98 101 103  99]

Enter the User Id : 123
5/5 [==============================] - 0s 3ms/step
[142  15  40  47  71  43  16  29  99 116]


In [16]:
# import pandas as pd

# song_df = pd.read_excel("/content/song_list.xlsx")

# song_df.head()

In [22]:
user_id = int(input("Enter the User Id"))
results = recommedor(user_id)

print(results)

# for result in results:
#   songs = song_df.iloc[result].values[0]
#   print(songs)

# # [145  36  85 106 111]

Enter the User Id144
5/5 [==============================] - 0s 3ms/step
[116  15  29  43  47  71  99 114 142  40]


In [23]:
import pickle

model_file_path = 'song_recommender.pkl'

with open(model_file_path, 'wb') as file:
    pickle.dump(model, file)

print("Model saved successfully as", model_file_path)

Model saved successfully as song_recommender.pkl
